# Fake News Detection

### Importing Libraries

In [4]:
import numpy as np # type: ignore
import pandas as pd # type: ignore
import matplotlib.pyplot as plt # type: ignore
import seaborn as sns # type: ignore

#re = regular expression(Useful for searching the text in a document)
import re
#nltk => Natural Language toolkit
#corpus => body of the text
#Stop words are those words, that don't add much value to the text(a,an,the,what,where)
from nltk.corpus import stopwords # type: ignore
#PorterStemmer is used for stemming(Removes the prefix and suffix of the word and returns the root word for it)
from nltk.stem.porter import PorterStemmer # type: ignore
#TfidfVectorizer is used to convert the words into feature vectors(Numbers)
from sklearn.feature_extraction.text import TfidfVectorizer # type: ignore

from sklearn.linear_model import LogisticRegression # type: ignore
from sklearn.model_selection import train_test_split # type: ignore
from sklearn.metrics import accuracy_score # type: ignore

import nltk # type: ignore
nltk.download("stopwords")

[nltk_data] Error loading stopwords: <urlopen error [SSL:
[nltk_data]     CERTIFICATE_VERIFY_FAILED] certificate verify failed:
[nltk_data]     unable to get local issuer certificate (_ssl.c:1000)>


False

In [3]:
#Printing the stopwords in english
print(stopwords.words('english'))

LookupError: 
**********************************************************************
  Resource [93mstopwords[0m not found.
  Please use the NLTK Downloader to obtain the resource:

  [31m>>> import nltk
  >>> nltk.download('stopwords')
  [0m
  For more information see: https://www.nltk.org/data.html

  Attempted to load [93mcorpora/stopwords[0m

  Searched in:
    - '/Users/pushpakreddy/nltk_data'
    - '/Library/Frameworks/Python.framework/Versions/3.12/nltk_data'
    - '/Library/Frameworks/Python.framework/Versions/3.12/share/nltk_data'
    - '/Library/Frameworks/Python.framework/Versions/3.12/lib/nltk_data'
    - '/usr/share/nltk_data'
    - '/usr/local/share/nltk_data'
    - '/usr/lib/nltk_data'
    - '/usr/local/lib/nltk_data'
**********************************************************************


### Loading the dataset

### Loading the dataset

In [ ]:
data = pd.read_csv("dataset.csv")
data.head()

,id,title,author,text,label
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1


In [ ]:
#Shape of the dataset
data.shape

(20800, 5)

### Data Preprocessing

In [ ]:
#Checking for missing values
data.isnull().sum()

id           0
title      558
author    1957
text        39
label        0
dtype: int64

In [ ]:
#Handling missing values
#Replacing the missing values with empty strings
data = data.fillna('')

In [ ]:
data.isnull().sum()

id        0
title     0
author    0
text      0
label     0
dtype: int64

In [ ]:
# Merging the author name and news title to form the input
data['content'] = data["author"] + ' ' + data["title"]

In [ ]:
data['content']

0        Darrell Lucus House Dem Aide: We Didn’t Even S...
1        Daniel J. Flynn FLYNN: Hillary Clinton, Big Wo...
2        Consortiumnews.com Why the Truth Might Get You...
3        Jessica Purkiss 15 Civilians Killed In Single ...
4        Howard Portnoy Iranian woman jailed for fictio...
                               ...                        
20795    Jerome Hudson Rapper T.I.: Trump a ’Poster Chi...
20796    Benjamin Hoffman N.F.L. Playoffs: Schedule, Ma...
20797    Michael J. de la Merced and Rachel Abrams Macy...
20798    Alex Ansary NATO, Russia To Hold Parallel Exer...
20799              David Swanson What Keeps the F-35 Alive
Name: content, Length: 20800, dtype: object

In [ ]:
#To make predictions we will be using the content column as features and labels as targets
#Seperating the data and label
x = data.drop(columns = 'label',axis = 1)
y = data['label']

In [ ]:
x.head()

,id,title,author,text,content
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus House Dem Aide: We Didn’t Even S...
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,"Daniel J. Flynn FLYNN: Hillary Clinton, Big Wo..."
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",Consortiumnews.com Why the Truth Might Get You...
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,Jessica Purkiss 15 Civilians Killed In Single ...
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,Howard Portnoy Iranian woman jailed for fictio...


In [ ]:
y.head()

0    1
1    0
2    1
3    1
4    1
Name: label, dtype: int64

### Stemming Procedure
#### Stemming is the process of reducing a word to its Root Word
##### Eg : actor, actress, acting ==> act

In [ ]:
#Using Porter Stemmer
port_stem = PorterStemmer()

In [ ]:
def stemming(content):
    stemmed_content = re.sub('[^a-zA-Z]',' ',content)
    stemmed_content = stemmed_content.lower()
    stemmed_content = stemmed_content.split()
    stemmed_content = [port_stem.stem(word) for word in stemmed_content if not word in stopwords.words('english')]
    stemmed_content = ' '.join(stemmed_content)
    return stemmed_content

### Before Stemming

In [ ]:
data['content'].head()

0    Darrell Lucus House Dem Aide: We Didn’t Even S...
1    Daniel J. Flynn FLYNN: Hillary Clinton, Big Wo...
2    Consortiumnews.com Why the Truth Might Get You...
3    Jessica Purkiss 15 Civilians Killed In Single ...
4    Howard Portnoy Iranian woman jailed for fictio...
Name: content, dtype: object

### Stemming

In [ ]:
data['content'] = data['content'].apply(stemming)

LookupError: 
**********************************************************************
  Resource [93mstopwords[0m not found.
  Please use the NLTK Downloader to obtain the resource:

  [31m>>> import nltk
  >>> nltk.download('stopwords')
  [0m
  For more information see: https://www.nltk.org/data.html

  Attempted to load [93mcorpora/stopwords[0m

  Searched in:
    - '/Users/pushpakreddy/nltk_data'
    - '/Library/Frameworks/Python.framework/Versions/3.12/nltk_data'
    - '/Library/Frameworks/Python.framework/Versions/3.12/share/nltk_data'
    - '/Library/Frameworks/Python.framework/Versions/3.12/lib/nltk_data'
    - '/usr/share/nltk_data'
    - '/usr/local/share/nltk_data'
    - '/usr/lib/nltk_data'
    - '/usr/local/lib/nltk_data'
**********************************************************************


### After Stemming

In [ ]:
print(data['content'])

0        darrel lucu hou dem aid even see comey letter ...
1        daniel j flynn flynn hillari clinton big woman...
2                   consortiumnew com truth might get fire
3        jessica purkiss civilian kill singl us airstri...
4        howard portnoy iranian woman jail fiction unpu...
                               ...                        
20795    jerom hudson rapper trump poster child white s...
20796    benjamin hoffman n f l playoff schedul matchup...
20797    michael j de la merc rachel abram maci said re...
20798    alex ansari nato russia hold parallel exerci b...
20799                            david swanson keep f aliv
Name: content, Length: 20800, dtype: object


In [ ]:
#Since text is huge, we are not using it as a feature
# Seperating the data and label
x = data['content'].values
y = data['label'].values

KeyError: 'content'

In [ ]:
print(x)

['darrel lucu hou dem aid even see comey letter jason chaffetz tweet'
 'daniel j flynn flynn hillari clinton big woman campu breitbart'
 'consortiumnew com truth might get fire' ...
 'michael j de la merc rachel abram maci said receiv takeov approach hudson bay new york time'
 'alex ansari nato russia hold parallel exerci balkan'
 'david swanson keep f aliv']


In [ ]:
print(y)

[1 0 1 ... 0 1 1]


In [ ]:
y.shape

(20800,)

In [ ]:
x.shape

(20800,)

### Converting the text of content into numerical data

In [ ]:
# converting the textual data to numerical data
vectorizer = TfidfVectorizer()
vectorizer.fit(x)

x = vectorizer.transform(x)

NameError: name 'x' is not defined

In [ ]:
print(x)

  (0, 15553)	0.28485063562728646
  (0, 13348)	0.2565896679337957
  (0, 8822)	0.3635963806326075
  (0, 8546)	0.29212514087043684
  (0, 7612)	0.24785219520671603
  (0, 6930)	0.21874169089359144
  (0, 4917)	0.233316966909351
  (0, 3759)	0.2705332480845492
  (0, 3567)	0.3598939188262559
  (0, 2933)	0.2468450128533713
  (0, 2464)	0.3676519686797209
  (0, 263)	0.27010124977708766
  (1, 16656)	0.3025156488372128
  (1, 6744)	0.19152496072048605
  (1, 5440)	0.7186013955384664
  (1, 3535)	0.2653147533915268
  (1, 2790)	0.19208753385709676
  (1, 2206)	0.36915639258038363
  (1, 1877)	0.15614790568229528
  (1, 1481)	0.2957471154505952
  (2, 15478)	0.41544962664721613
  (2, 9532)	0.49351492943649944
  (2, 5903)	0.3474613386728292
  (2, 5326)	0.3866530551182615
  (2, 3072)	0.46097489583229645
  :	:
  (20797, 12998)	0.2483705036831893
  (20797, 12227)	0.27276402145717243
  (20797, 12022)	0.24790022252744132
  (20797, 10216)	0.0804189541935242
  (20797, 9500)	0.17463635692029988
  (20797, 9431)	0.29394

### Splitting our data into training and testing data

In [ ]:
x_train,x_test,y_train,y_test = train_test_split(x,y,test_size = 0.2,random_state = 2,stratify = y)
#Stratify makes sure that the proportion of real news vs fake news in the real data set is maintained in the train test split

NameError: name 'x' is not defined

In [ ]:
print(y.shape,y_train.shape,y_test.shape)

(20800,) (16640,) (4160,)


### Logistic Regression Model

In [ ]:
model = LogisticRegression()
model.fit(x_train,y_train)

LogisticRegression()

### Evaluation

In [ ]:
pred = model.predict(x_test)
acc = accuracy_score(pred,y_test)
print(acc)

0.9790865384615385


### Making a predictive system

In [ ]:
x_new = x_test[1]

prediction = model.predict(x_new)
print(prediction,y_test[0])
if prediction == 1:
    print("The news is true")
else:
    print("The news is fake")

NameError: name 'x_test' is not defined

In [ ]:
import pickle

with open('fake_news_detect.pkl', 'wb') as f:
  pickle.dump(model, f)


In [ ]:
with open('fake_news_detect.pkl', 'rb') as f:
  loaded_model = pickle.load(f)


In [ ]:
def preprocess_text(text):
  text = text.lower()

  tokens = text.split()

  from nltk.corpus import stopwords
  stop_words = set(stopwords.words('english'))
  port_stem = PorterStemmer()
  def stemming(text):
    stemmed_content = re.sub('[^a-zA-Z]',' ',text)
    stemmed_content = stemmed_content.lower()
    stemmed_content = stemmed_content.split()
    stemmed_content = [port_stem.stem(word) for word in stemmed_content if not word in stopwords.words('english')]
    stemmed_content = ' '.join(stemmed_content)
    return stemmed_content

  preprocessed_text = ' '.join(tokens)
  return preprocessed_text


In [ ]:
!pip install gradio scikit-learn nltk


In [ ]:
def preprocess_text(text):
  text = text.lower()

  tokens = text.split()

  from nltk.corpus import stopwords
  stop_words = set(stopwords.words('english'))
  port_stem = PorterStemmer()
  def stemming(text):
    stemmed_content = re.sub('[^a-zA-Z]',' ',text)
    stemmed_content = stemmed_content.lower()
    stemmed_content = stemmed_content.split()
    stemmed_content = [port_stem.stem(word) for word in stemmed_content if not word in stopwords.words('english')]
    stemmed_content = ' '.join(stemmed_content)
    return stemmed_content

  preprocessed_text = ' '.join(tokens)
  return preprocessed_text


In [ ]:
import pickle
#Here I am loading the model
with open('/content/my_model.pkl', 'rb') as f:
  loaded_model = pickle.load(f)

In [ ]:
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer

def preprocess_text(text):
  text = text.lower()
  tokens = text.split()
  stop_words = set(stopwords.words('english'))
  stemmer = PorterStemmer()

  # Combine stemming and stop word removal:
  preprocessed_text = [stemmer.stem(word) for word in tokens if word not in stop_words]
  return ' '.join(preprocessed_text)


In [ ]:
def predict_news(text):
  preprocessed_text = preprocess_text(text)
  prediction = loaded_model.predict([preprocessed_text])
  if prediction[0] > 0.5:
    return "Real"
  else:
    return "Fake"


In [ ]:
import gradio as gr  #code for gradio interface

interface = gr.Interface(
  fn=predict_news,
  inputs="textbox",
  outputs="text",
  title="Fake News Detector",
  description="Enter a news article to see if it's likely real or fake."
)

interface.launch()
